# Plotting resampled figures and calculating spatial consistency

### Author: Chris Wyburn-Powell, [github](https://github.com/chrisrwp/synthetic-ensemble/SIC/Resampled_figures_SIC.ipynb)

**Input**: <br>
- Resampled models and observations

**Output**: <br>
- Figures of ratio and difference between the three measures of variability **$\sigma_{LE}$**,  **$\sigma_{mem}$**, **$\sigma_{obs}$**
- Consistency figure of all 6 models with NSIDC CDR and HadISST1, alongside consistency agreement and disagreement between the observational datasets

In [1]:
import numpy as np
import xarray as xr
import datetime
import math as m
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.path as mpath
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
import matplotlib.patheffects as pe
import cartopy.crs as ccrs

print(datetime.datetime.utcnow().strftime("%H:%M UTC %a %Y-%m-%d"))

22:51 UTC Fri 2021-09-17


In [2]:
data_path = '/glade/scratch/cwpowell/Synthetic_ensemble/'

model_names  = ['CanESM2', 'CESM1', 'CSIRO_MK36', 'GFDL_CM3', 'GFDL_ESM2M', 'MPI_ESM1']
model_print_names  = ['CanESM2', 'CESM1', 'CSIRO MK3.6', 'GFDL CM3', 'GFDL ESM2M', 'MPI ESM1' ]
dataset_names = ['HadISST1', 'Merged_Hadley_OI', 'NSIDC_BT', 'NSIDC_CDR', 'NSIDC_NT']

label_letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i' , 'j', 'k', 'l']

# Make land masks

In [4]:
# Make >30N areacello files for regridding

# for model_name in model_names:
#     if model_name == 'MPI_ESM1':
#         lat_name = 'latitude'
#     else:
#         lat_name = 'lat'
        
#     areacello = xr.open_dataset(data_path+'Raw_data/areacello/areacello_{}.nc'.format(model_name))
#     areacello_30N = areacello.where(areacello[lat_name]>30, drop=True)
#     areacello_30N.to_netcdf(data_path+'Raw_data/areacello/areacello_{}_30N.nc'.format(model_name))

In [132]:
#quickly make mask from regridded 30N areacello files. Land = np.nan, ocean = 0
masks = {}
masks_NSIDC = {}

NSIDC_sample = xr.open_dataset(data_path+'Raw_data/observations/NSIDC_CDR_v4/seaice_conc_monthly_nh_202003_f17_v04r00_regrid.nc')

for model_name in model_names:
    if model_name == 'GFDL_CM3':
        GFDL_CM3_SIC_regrid = xr.open_dataset(data_path+'Raw_data/areacello/GFDL_CM3/GFDL_CM3_1979_03_mem1_regrid.nc')
        mask_temp = GFDL_CM3_SIC_regrid['sic'].where(GFDL_CM3_SIC_regrid['sic']>0,-999)
    else:
        areacello_regrid = xr.open_dataset(data_path+'Raw_data/areacello/areacello_{}_30N_regrid.nc'.format(model_name))
        mask_temp = areacello_regrid['areacello'].where(areacello_regrid['areacello']>0,-999)
        
    mask_temp = mask_temp.where(mask_temp==-999,0)
    masks[model_name] = mask_temp.where(mask_temp==0)
    
    masks_NSIDC[model_name] = mask_temp.where(mask_temp==0).where(NSIDC_sample['cdr_seaice_conc_monthly'][0]>-1)
    
#GFDL models have partial land cells so masks don't work well, use CESM1 instead
masks_copied = masks.copy()
masks_copied['GFDL_CM3'] = masks['CESM1']
masks_copied['GFDL_ESM2M'] = masks['CESM1']

masks_NSIDC['GFDL_CM3'] = masks_NSIDC['CESM1']
masks_NSIDC['GFDL_ESM2M'] = masks_NSIDC['CESM1']

# Plot ratio and differences with cartopy

In [4]:
def two_by_six_plots(data, masks, colors, levels_, cbar_levels_, cbar_label, white=False, extend_kw=False, save_name=False):
        
    yrs = ['', '', '', '', '', '', '', '', '', '', '', '']
    
    fig = plt.figure(figsize=[15.8,5.5])
    
    if white: mpl.rcParams['text.color'] = '0' #white titles and text
    
    for i in range(12):
        if type(data[i]) != str:
            if i < 6: #march
                lat_min = 50
            else: #september
                lat_min = 60

            ax = fig.add_subplot(2,6,i+1, projection = ccrs.NorthPolarStereo(central_longitude=0))
            ax.set_extent((-180,180,int(lat_min),90), ccrs.PlateCarree())

            theta = np.linspace(0, 2*np.pi, 100) #make the plot circular
            center, radius = [0.5, 0.5], 0.5
            verts = np.vstack([np.sin(theta), np.cos(theta)]).T
            circle = mpath.Path(verts * radius + center)
            ax.set_boundary(circle, transform=ax.transAxes)

            if masks:
                data_i = data[i].where(masks[model_names[i%6]]>-9)
            else:
                data_i = data[i]
            
            masks[model_names[i%6]].plot(ax=ax, cmap='binary', transform=ccrs.PlateCarree(), levels=np.arange(-0.01, 100, 1), add_colorbar=False)
            
            plotting = data_i.plot(ax=ax, cmap=str(colors), add_colorbar=False, transform=ccrs.PlateCarree(), levels=levels_, add_labels=False)

            ax.coastlines()
            ax.gridlines()
            ax.set_title(model_print_names[i%6], fontsize=14, fontweight='bold')
            ax.set_facecolor('0.5')



            if i == 0:
                ax.text(0.14, 0.95, 'Mar', fontsize=16, fontweight='bold', transform=ax.transAxes, va='top', ha='right');
            elif i == 6:
                ax.text(0.14, 0.95, 'Sep', fontsize=16, fontweight='bold', transform=ax.transAxes, va='top', ha='right');

                fig.subplots_adjust(right=0.89)
        cbar_ax = fig.add_axes([0.905, 0.17, 0.02, 0.8])
    
    if extend_kw:
        cb = fig.colorbar(plotting, cax=cbar_ax, ticks=cbar_levels_, spacing='uniform', extend=extend_kw, orientation='vertical')
    else:
        cb = fig.colorbar(plotting, cax=cbar_ax, ticks=cbar_levels_, spacing='uniform', extend='both', orientation='vertical')
    
    
    if white:
        cbtick_obj = plt.getp(cb.ax.axes, 'yticklabels')   
        plt.setp(cbtick_obj, color='1')
        cb.ax.set_ylabel(str(cbar_label), fontsize=18, color='1') 
    else:
        cb.ax.set_ylabel(str(cbar_label), fontsize=18, color='0') 
    
#     fig.suptitle(cbar_label, fontsize=23)
    fig.subplots_adjust(top=0.99, hspace=-0.1, wspace=0.04)
    fig.tight_layout()
    
    if save_name:
        fig.savefig('/glade/scratch/cwpowell/Synthetic_ensemble/SIC/figures/{}.pdf'.format(save_name), bbox_inches = 'tight', pad_inches = 0)
        fig.savefig('/glade/scratch/cwpowell/Synthetic_ensemble/SIC/figures/{}.png'.format(save_name), dpi=400, bbox_inches = 'tight', pad_inches = 0)

# $\sigma_{mem}$/$\sigma_{LE}$

In [18]:
#all for individual detrending, change '-'or '/'for diff or ratio
mem_LE = []
mem_ens = []
mem_ind = []
mem_obs_CDR      = []
mem_obs_NT       = []
mem_obs_BT       = []
mem_obs_HadISST1 = []
mem_obs_Merged   = []

mem_ens_ind = []
mem_obs_CDR_ens_jak = []
ens_ind_diff_mem_obs_CDR = []

CDR = xr.open_dataset(data_path+'SIC/Resampled/NSIDC_CDR_resampled_adj_individual_1979_2020_03_09_regrid.nc')
# CDR_jak = xr.open_dataset(data_path+'SIC/Resampled/NSIDC_CDR_resampled_adj_jackknife_1979_2020_03_09_regrid.nc')
NT  = xr.open_dataset(data_path+'SIC/Resampled/NSIDC_NT_resampled_adj_individual_1979_2020_03_09_regrid.nc')
BT  = xr.open_dataset(data_path+'SIC/Resampled/NSIDC_BT_resampled_adj_individual_1979_2020_03_09_regrid.nc')
HadISST1 = xr.open_dataset(data_path+'SIC/Resampled/HadISST1_resampled_adj_individual_1979_2020_03_09_regrid.nc')
Merged   = xr.open_dataset(data_path+'SIC/Resampled/Merged_Hadley_OI_resampled_adj_individual_1979_2020_03_09_regrid.nc')

for month_ in [3,9]:
    for model_i, model_name in enumerate(model_names):
        LE = xr.open_dataset(data_path+'SIC/Detrended/Sigma_LE_{}_regrid.nc'.format(model_name))
        sigma_LE = LE['adj_individual'].sel(month=month_)
        
        mem = xr.open_dataset(data_path+'SIC/Resampled/{}_resampled_{}_adj_individual_regrid.nc'.format(model_name, str(month_).zfill(2)))
        mem_e = xr.open_dataset(data_path+'SIC/Resampled/{}_resampled_{}_adj_ensemble_regrid.nc'.format(model_name, str(month_).zfill(2)))
        sigma_mem = mem['SD'].mean('member')
        
        obs_CDR = CDR['SD'].sel(month=month_)*100
        obs_NT  = NT['SD'].sel(month=month_)*100
        obs_BT  = BT['SD'].sel(month=month_)*100
        obs_HadISST1 = HadISST1['SD'].sel(month=month_)*100
        obs_Merged   = Merged['SD'].sel(month=month_)*100
        
        mem_ens.append(mem_e['SD'].mean('member'))
        mem_ind.append(sigma_mem)
        
        #do the ratio or difference calculation
        mem_LE.append(sigma_LE - sigma_mem)
        mem_obs_CDR.append(obs_CDR - sigma_mem)
        mem_obs_NT.append(obs_NT / sigma_mem)
        mem_obs_BT.append(obs_BT / sigma_mem)
        mem_obs_HadISST1.append(obs_HadISST1 - sigma_mem)
        mem_obs_Merged.append(obs_Merged / sigma_mem)
        
        mem_ens_ind.append(mem_e['SD'].mean('member') / (sigma_mem*100))
        
        mem_obs_CDR_ens_jak.append(CDR_jak['SD'].sel(month=month_) / sigma_mem)
        
#         ens_ind_diff_mem_obs_CDR.append((CDR_jak['SD'].sel(month=month_) / (mem_e['SD'].mean('member')) / (obs_CDR / sigma_mem))

In [141]:
two_by_six_plots(mem_LE, masks_copied, 'RdBu', np.arange(-2.001,2.01,0.1), np.arange(-2,2.01,0.25), '$\sigma_{LE} - \sigma_{mem}$', extend_kw='max', save_name='Resampled_figures_SIC_LE_mem_diff_tight') #use 0.001 offset for CSIRO rounding errors
# two_by_six_plots(mem_LE, masks_copied, 'RdBu', np.arange(0,2.01,0.1), np.arange(0,2.01,0.1), '$\sigma_{LE} \ / \ \sigma_{mem}$', extend_kw='max', save_name='Resampled_figures_SIC_LE_mem_ratio_tight')

In [140]:
two_by_six_plots(mem_obs_CDR, masks_copied, 'RdBu', np.arange(-2,2.01,0.1), np.arange(-2,2.01,0.25), '$\sigma_{obs} - \sigma_{mem}$', extend_kw='max', save_name='Resampled_figures_SIC_obs_CDR_mem_diff') 
# two_by_six_plots(mem_obs_HadISST1, masks_copied, 'RdBu', np.arange(0,2.01,0.1), np.arange(0,2.01,0.1), '$\sigma_{obs} \ / \ \sigma_{mem}$', extend_kw='max', save_name='Resampled_figures_SIC_obs_HadISST1_mem_ratio')

In [138]:
#change between ensemble and individual 
two_by_six_plots(ens_ind_diff_mem_obs_CDR, masks_copied, 'RdBu', np.arange(0.85,1.16,0.01), np.arange(0,2.01,0.05), '$\sigma_{obs} \ / \ \sigma_{mem} \ Ensemble \ / \ Individual$', extend_kw='both')#, save_name='Resampled_figures_SIC_LE_mem_ratio')

# Comparisons between different detrending

In [21]:
#sigma mem comparisons
adj_ens_ens = []
adj_ind_ind = []
adj_end_adj_ind = []
ens_ind = []

for month_ in [3,9]:
    for model_i, model_name in enumerate(model_names):
        adj_ens = xr.open_dataset(data_path+'SIC/Resampled/{}_resampled_{}_adj_ensemble_regrid.nc'.format(model_name, str(month_).zfill(2)))
        ens     = xr.open_dataset(data_path+'SIC/Resampled/{}_resampled_{}_ensemble_regrid.nc'.format(model_name, str(month_).zfill(2)))
        adj_ind = xr.open_dataset(data_path+'SIC/Resampled/{}_resampled_{}_adj_individual_regrid.nc'.format(model_name, str(month_).zfill(2)))
        ind     = xr.open_dataset(data_path+'SIC/Resampled/{}_resampled_{}_individual_regrid.nc'.format(model_name, str(month_).zfill(2)))
        
        adj_ens_ens.append(adj_ens['SD'].mean('member') - ens['SD'].mean('member'))
        adj_ind_ind.append(adj_ind['SD'].mean('member') - ind['SD'].mean('member'))
        adj_end_adj_ind.append(adj_ens['SD'].mean('member') - adj_ind['SD'].mean('member'))
        ens_ind.append(ens['SD'].mean('member') - ind['SD'].mean('member'))

In [22]:
#sigma mem sigma LE
LE_adj_ens = []
LE_ens = []

LE_adj_ens_LE_ens = []
LE_ens_LE_ind = []

LE_adj_ind = []
LE_ind = []

for month_ in [3,9]:
    for model_i, model_name in enumerate(model_names):
        adj_ens = xr.open_dataset(data_path+'SIC/Resampled/{}_resampled_{}_adj_ensemble_regrid.nc'.format(model_name, str(month_).zfill(2)))
        ens     = xr.open_dataset(data_path+'SIC/Resampled/{}_resampled_{}_ensemble_regrid.nc'.format(model_name, str(month_).zfill(2)))
        adj_ind = xr.open_dataset(data_path+'SIC/Resampled/{}_resampled_{}_adj_individual_regrid.nc'.format(model_name, str(month_).zfill(2)))
        ind     = xr.open_dataset(data_path+'SIC/Resampled/{}_resampled_{}_individual_regrid.nc'.format(model_name, str(month_).zfill(2)))
        
        LE = xr.open_dataset(data_path+'SIC/Detrended/Sigma_LE_{}_regrid.nc'.format(model_name))
        
        LE_adj_ens.append(adj_ens['SD'].mean('member') / LE['adj_ensemble'].sel(month=month_))
        LE_ens.append(ens['SD'].mean('member') / LE['ensemble'].sel(month=month_))
        LE_adj_ind.append(adj_ind['SD'].mean('member') / LE['adj_individual'].sel(month=month_))
        LE_ind.append(ind['SD'].mean('member') / LE['individual'].sel(month=month_))
        
        LE_adj_ens_LE_ens.append(adj_ens['SD'].mean('member') / LE['adj_ensemble'].sel(month=month_) - ens['SD'].mean('member') / LE['ensemble'].sel(month=month_))
        
        LE_ens_LE_ind.append(ens['SD'].mean('member') / LE['ensemble'].sel(month=month_) - ind['SD'].mean('member') / LE['individual'].sel(month=month_))

In [35]:
#difference ens minus ind
LE_ens_ind = []
mem_ens_ind = []
for month_ in [3,9]:
    for model_i, model_name in enumerate(model_names):
        LE = xr.open_dataset(data_path+'SIC/Detrended/Sigma_LE_{}_regrid.nc'.format(model_name))
        
        adj_ens = xr.open_dataset(data_path+'SIC/Resampled/{}_resampled_{}_adj_ensemble_regrid.nc'.format(model_name, str(month_).zfill(2)))
        adj_ind = xr.open_dataset(data_path+'SIC/Resampled/{}_resampled_{}_adj_individual_regrid.nc'.format(model_name, str(month_).zfill(2)))
        
        LE_ens_ind.append(LE['adj_ensemble'].sel(month=month_) / LE['adj_individual'].sel(month=month_))
        mem_ens_ind.append(adj_ens['SD'].mean('member') / adj_ind['SD'].mean('member'))

In [142]:
two_by_five_plots(jak_ind, 'RdBu', np.arange(0.90,1.101,0.01), np.arange(0.90,1.11,0.05), r'$\sigma_{obs}$'+'  Jackknife / Individual', title_list_, extend_kw='neither')

# Consistency

In [136]:
#plot example of envalope of members and observations
mean_SD = 'SD'
month_ = 9
model_name = 'MPI_ESM1'
i = -6

sigma_mem = xr.open_dataset(data_path+'SIC/Resampled/{}_resampled_0{}_adj_individual_regrid.nc'.format(model_name, month_))
sigma_mem = sigma_mem[mean_SD]

for obs_data in ['NSIDC_CDR','HadISST1']:
    print(obs_data)
    sigma_obs = xr.open_dataset(data_path+'SIC/Resampled/{}_resampled_adj_individual_1979_2020_03_09_regrid.nc'.format(obs_data))
    sigma_obs = sigma_obs[mean_SD].sel(month=month_)*100


    sigma_obs.isel(lat=i).plot(label=obs_data, c='r')
    plt.fill_between(sigma_mem['lon'], sigma_mem.min('member').isel(lat=i), sigma_mem.max('member').isel(lat=i),
                     facecolor='b', edgecolor='face', alpha=0.4)

plt.yscale('log')
plt.legend()
plt.title(model_name+', March, 84.9 N');

### Caclulate inconsistent areas and combine

In [13]:
def calc_inconsistent(mean_SD, obs_data, threshold):
    'Calculates whether there is at least one members overlapping with observations'
    
    too_high_flag_all  = []
    too_low_flag_all   = []
    no_sea_ice_all = []
    for model_name in model_names:

        too_high_flag = []
        too_low_flag  = []
        no_sea_ice = []
        for month_ in [3,9]:

            #load either the average across resamplings or the SD across resamplings of SD(time)
            sigma_mem = xr.open_dataset(data_path+'SIC/Resampled/{}_resampled_0{}_adj_individual_regrid.nc'.format(model_name, month_))
            sigma_mem = sigma_mem[mean_SD]

            if model_name == 'CSIRO_MK36': sigma_mem = sigma_mem.where(sigma_mem>1e-10,0) #CSIRO has e-11 rather than nans

            sigma_obs = xr.open_dataset(data_path+'SIC/Resampled/{}_resampled_adj_individual_1979_2020_03_09_regrid.nc'.format(obs_data))
            sigma_obs = sigma_obs[mean_SD].sel(month=month_)*100
            #create an array of ones of the shape of the lat/lon grid
            ones_array = xr.DataArray(data   = np.ones([len(sigma_obs['lat']),len(sigma_obs['lon'])]),
                                      coords = {'lat':sigma_obs['lat'], 'lon':sigma_obs['lon']},
                                      dims   = {'lat':sigma_obs['lat'], 'lon':sigma_obs['lon']})

            #find the difference of the minimum and maximum values from observations 
            mins = sigma_mem.min('member') - (sigma_obs)*(1+threshold) #negative number is good
            maxs = sigma_mem.max('member') - (sigma_obs)*(1-threshold) #positive number is good

            #too_high_flag = 1: bad, all model members too high. too_high_flag = 0: good, at least one member below obs
            #too_low_flag  = 1: bad, all model members too low.  too_high_flag = 0: good, at least one member above obs
            too_high_flag.append(ones_array.copy().where(mins > 0, 0))
            too_low_flag.append(ones_array.copy().where(maxs < 0, 0))
            
            
            ##### find regions where no sea ice exists (or pole hole)
            no_sea_ice_mem = sigma_mem.mean('member').where(sigma_mem.mean('member')>-1,-1)
            no_sea_ice_mem = no_sea_ice_mem.where(no_sea_ice_mem==-1,0)
            
            no_sea_ice_obs = sigma_obs.where(sigma_obs>-1,-1)
            no_sea_ice_obs = no_sea_ice_obs.where(no_sea_ice_obs==-1,0)

            no_sea_ice_both = xr.ufuncs.logical_or(no_sea_ice_obs, no_sea_ice_mem)
            no_sea_ice.append(no_sea_ice_both.where(no_sea_ice_both==1,0))

        
        too_high_flag_all.append(xr.concat((too_high_flag), dim='month'))
        too_low_flag_all.append(xr.concat((too_low_flag), dim='month'))
        no_sea_ice_all.append(xr.concat((no_sea_ice), dim='month'))
           
    too_high_flag_all = xr.concat((too_high_flag_all), dim='model')
    too_high_flag_all['model'] = model_names
    too_high_flag_all['month'] = [3,9]
        
    too_low_flag_all = xr.concat((too_low_flag_all), dim='model')
    too_low_flag_all['model'] = model_names
    too_low_flag_all['month'] = [3,9]
    
    no_sea_ice_all = xr.concat((no_sea_ice_all), dim='model')
    no_sea_ice_all['model'] = model_names
    no_sea_ice_all['month'] = [3,9]
    
    return too_high_flag_all, too_low_flag_all, no_sea_ice_all

In [133]:
#run the inconsistent calculations
CDR_SD = calc_inconsistent('SD', 'NSIDC_CDR', 0)
CDR_mean = calc_inconsistent('mean', 'NSIDC_CDR', 0)

Had_SD = calc_inconsistent('SD', 'HadISST1', 0)
Had_mean = calc_inconsistent('mean', 'HadISST1', 0)

#compute subsequent calculations
red_CDR = CDR_SD[0] * CDR_mean[0]
red_Had = Had_SD[0] * Had_mean[0]

blue_CDR = CDR_SD[1] * CDR_mean[1]
blue_Had = Had_SD[1] * Had_mean[1]

black_CDR = (red_CDR + blue_CDR).where((red_CDR + blue_CDR)==2,0)
black_Had = (red_Had + blue_Had).where((red_Had + blue_Had)==2,0)

red_both  = (red_CDR+red_Had).where((red_CDR+red_Had)==2,0)/2
blue_both = (blue_CDR+blue_Had).where((blue_CDR+blue_Had)==2,0)/2

red_disagree  = (red_CDR+red_Had).where((red_CDR+red_Had)==1,0)
blue_disagree = (blue_CDR+blue_Had).where((blue_CDR+blue_Had)==1,0)
black_both = xr.ufuncs.logical_not(xr.ufuncs.isnan((red_disagree + blue_disagree).where((red_disagree + blue_disagree)>0)))

no_ice_both = xr.ufuncs.logical_not(xr.ufuncs.isnan((CDR_SD[2] + Had_SD[2]).where((CDR_SD[2] + Had_SD[2])>0)))

#make lists for iterating through when plotting
red_list    = [red_CDR, red_Had, red_both]
blue_list   = [blue_CDR, blue_Had, blue_both]
black_list  = [black_CDR, black_Had, black_both]
no_ice_list = [CDR_SD[2], Had_SD[2], no_ice_both]

In [134]:
model_names_center = ['   CanESM2', 
                      '     CESM1', 
                      ' CSIRO MK36',
                      '  GFDL CM3',
                      'GFDL ESM2M', 
                      '  MPI ESM1']

titles_datasets = ['           CDR', '      HadISST1', '          Both']

In [143]:
#plot 6x6
fig = plt.figure(figsize=[18,18])

for dataset in range(3):
    
    for model_i, model_name in enumerate(model_names):
        for month_i, month_ in enumerate([3,9]):

            i = (model_i*2) + month_i + 1
            if month_ == 3:
                lat_min = 50
            else:
                lat_min = 60

            ax = fig.add_subplot(6,6,i+(model_i*4 + dataset*2), projection = ccrs.NorthPolarStereo(central_longitude=0))
            ax.set_extent((-180,180,lat_min,90), ccrs.PlateCarree())

            #set boundary of subplots
            theta = np.linspace(0, 2*np.pi, 100) #make the plot circular
            center, radius = [0.5, 0.5], 0.5
            verts = np.vstack([np.sin(theta), np.cos(theta)]).T
            circle = mpath.Path(verts * radius + center)
            ax.set_boundary(circle, transform=ax.transAxes)
            ax.set_facecolor('1')
            
            #plot in sequence: red (too high), blue (too low), black (disagreement), baige (no sea ice), gray (land)
            #red - too high
            red_list[dataset].sel(model=model_name).sel(month=month_).plot(cmap='Reds', levels=np.arange(0.2,1.4,0.1), add_labels=False, add_colorbar=False, transform=ccrs.PlateCarree(), extend='neither')
            #blue - too low
            blue_list[dataset].sel(model=model_name).sel(month=month_).plot(cmap='Blues', levels=np.arange(0.2,1.4,0.1), add_labels=False, add_colorbar=False, transform=ccrs.PlateCarree(), extend='neither')
            #black - disagreement
            black_list[dataset].sel(model=model_name).sel(month=month_).plot(cmap='binary', levels=np.arange(0.9,1.11,0.1), add_labels=False, add_colorbar=False, transform=ccrs.PlateCarree(), extend='neither')            
            #baige - no sea ice 
            no_ice_list[dataset].sel(model=model_name).sel(month=month_).plot(cmap='Oranges', levels=np.arange(0.9,3,0.1), add_labels=False, add_colorbar=False, transform=ccrs.PlateCarree(), extend='neither')
            #gray - land 
            xr.ufuncs.isnan(masks_copied[model_name]).plot(cmap='binary', levels=np.arange(0.1,2.5,0.1), add_labels=False, add_colorbar=False, transform=ccrs.PlateCarree(), extend='neither')
            
            ax.coastlines()
            ax.gridlines()

            if i == 1:    
                ax.text(0.37, 1.14, titles_datasets[dataset], fontsize=22, fontweight='bold', transform=ax.transAxes, va='top', ha='left');
            elif i == 12:
                ax.text(0.2, 0, 'September', fontsize=18, fontweight='bold', transform=ax.transAxes, va='top', ha='left');
            elif i == 11:
                ax.text(0.3, 0, 'March', fontsize=18, fontweight='bold', transform=ax.transAxes, va='top', ha='left');
            
            if (i+(model_i*4 + dataset*2)-1) % 6 == 0:
                ax.text(-0.08, 0.16, model_names_center[model_i], fontsize=16, fontweight='bold', transform=ax.transAxes, va='bottom', ha='left', rotation='vertical');
                

fig.subplots_adjust(top=0.91, left=0.02, hspace=0.02, wspace=-0.4);
fig.savefig('/glade/scratch/cwpowell/Synthetic_ensemble/SIC/figures/Resampled_figures_SIC_Consistency_6x6.pdf', bbox_inches = 'tight', pad_inches = 0)
fig.savefig('/glade/scratch/cwpowell/Synthetic_ensemble/SIC/figures/Resampled_figures_SIC_Consistency_6x6.png', bbox_inches = 'tight', pad_inches = 0, dpi=300)